In [ ]:
import sys
from unittest import mock

# Mock Adafruit_DHT and RPi.GPIO if running on Windows
if sys.platform == "win32":
    # Mock the Adafruit_DHT library
    Adafruit_DHT = mock.Mock()
    Adafruit_DHT.read_retry = mock.Mock(return_value=(80, 25))  # Mock sensor values (humidity, temperature)

    # Mock the RPi.GPIO library
    GPIO = mock.Mock()
    GPIO.setmode.return_value = None
    GPIO.setup.return_value = None
    GPIO.output.return_value = None
    GPIO.cleanup.return_value = None

    print("Mocking Adafruit_DHT and RPi.GPIO for Windows")
else:
    # Import actual libraries when running on a Raspberry Pi or other compatible platforms
    import Adafruit_DHT
    import RPi.GPIO as GPIO
    print("Running on Raspberry Pi or another compatible system")

# Your original code below
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
import time

print('initialization...')
### initialization GPIO
relay_pin = 26
GPIO.setmode(GPIO.BCM)
GPIO.setup(relay_pin, GPIO.OUT)
sensor = Adafruit_DHT.DHT22  # DHT22 pin on Raspberry Pi
dht_pin = 23

######### INPUTS ########################
# Input Universe functions
temperature = np.arange(0, 11, 0.1)
humidity = np.arange(0, 11, 0.1)

# Input Membership Functions
temperature_cold = fuzz.gaussmf(temperature, 0, 1.5)
temperature_warm = fuzz.gaussmf(temperature, 5, 1.5)
temperature_hot = fuzz.gaussmf(temperature, 10, 1.5)

# Humidity
humidity_low = fuzz.trapmf(humidity, [0, 0, 1, 3])
humidity_high = fuzz.gaussmf(humidity, 10, 1.5)

########## OUTPUT ########################
# comfort
comfort = np.arange(0, 30, 0.1)
comfort_low = fuzz.trimf(comfort, [0, 5, 10])
comfort_ave = fuzz.trimf(comfort, [10, 15, 25])
comfort_very = fuzz.trimf(comfort, [20, 25, 30])

def temperature_category(temperature_in=18):
    temperature_cat_cold = fuzz.interp_membership(temperature, temperature_cold, temperature_in)
    temperature_cat_warm = fuzz.interp_membership(temperature, temperature_warm, temperature_in)
    temperature_cat_hot = fuzz.interp_membership(temperature, temperature_hot, temperature_in)
    return dict(cold=temperature_cat_cold, warm=temperature_cat_warm, hot=temperature_cat_hot)

def humidity_category(humidity_in=2):
    humidity_cat_low = fuzz.interp_membership(humidity, humidity_low, humidity_in)
    humidity_cat_high = fuzz.interp_membership(humidity, humidity_high, humidity_in)
    return dict(low=humidity_cat_low, high=humidity_cat_high)

print('saving membership...')
fig, ax = plt.subplots(2, 1)

# Plot temperature membership functions
lines_temp = ax[0].plot(temperature, temperature_cold, 'r', 
                        temperature, temperature_warm, 'm+', 
                        temperature, temperature_hot, 'b--')

# Set labels and titles for the temperature plot
ax[0].set_ylabel('Fuzzy membership')
ax[0].set_title('Temperature')
ax[0].set_ylim(-0.05, 1.05)
ax[0].set_xlim(0, 10)

# Create the legend for temperature plot
ax[0].legend([lines_temp[0], lines_temp[1], lines_temp[2]], 
             ['Temp. cold', 'Temp. warm', 'Temp. hot'], 
             loc='center left', bbox_to_anchor=(1, 0.5))

# Plot humidity membership functions
lines_humidity = ax[1].plot(humidity, humidity_low, 'r', 
                             humidity, humidity_high, 'b+')

ax[1].set_ylabel('Fuzzy membership')
ax[1].set_title('Humidity')
ax[1].set_ylim(-0.05, 1.05)
ax[1].set_xlim(0, 10)

# Create the legend for humidity plot
ax[1].legend([lines_humidity[0], lines_humidity[1]], 
             ['Hum. low', 'Hum. high'], 
             loc='center left', bbox_to_anchor=(1, 0.5))

plt.grid(True)
plt.tight_layout()

# Save the figure without needing lgd1 or lgd2
fig.savefig('fuzzy_mem_temp_hum.png', dpi=100, bbox_inches='tight')

print('done')

# sensing and make decision
print('program is ready for making decision based fuzzy logic')
machine_state = -1
try:
    while True:
        print('sensing...')
        sen_humidity, sen_temperature = Adafruit_DHT.read_retry(sensor, dht_pin)
        if sen_humidity is not None and sen_temperature is not None:
            print('Sensing: Temperature={0:0.1f}*C Humidity={1:0.1f}%'.format(sen_temperature, sen_humidity))
        else:
            # Simulate sensor values if mock is used
            sen_temperature = 18  # Mocked value
            sen_humidity = 80  # Mocked value

        # normalization
        norm_temperature = sen_temperature / 60.0
        norm_humidity = sen_humidity / 100.0
        print('Normalization: Temperature={0:0.0001f} Humidity={1:0.0001f}'.format(norm_temperature, norm_humidity))

        temp_in = temperature_category(norm_temperature)
        hum_in = humidity_category(norm_humidity)
        print('fuzzy membership: Temperature={0} Humidity={1}'.format(temp_in, hum_in))

        # Determine the weight and aggregate
        rule1 = np.fmax(temp_in['hot'], hum_in['low'])
        rule2 = temp_in['warm']
        rule3 = np.fmax(temp_in['warm'], hum_in['high'])
        imp1 = np.fmin(rule1, comfort_low)
        imp2 = np.fmin(rule2, comfort_ave)
        imp3 = np.fmin(rule3, comfort_very)
        aggregate_membership = np.fmax(imp1, np.fmax(imp2, imp3))

        # Defuzzify
        result_comfort = fuzz.defuzz(comfort, aggregate_membership, 'centroid')
        print(result_comfort)

        # make decision based on experiment
        if result_comfort >= 5.002:
            if machine_state < 0:
                machine_state = 1
                print("turn on a machine")
                GPIO.output(relay_pin, GPIO.HIGH)
            else:
                print("a machine already turn on")
        else:
            if machine_state > 0:
                machine_state = 0
                print("turn off a machine")
                GPIO.output(relay_pin, GPIO.LOW)
            else:
                print("a machine already turn off")
        time.sleep(2)

except KeyboardInterrupt:
    GPIO.output(relay_pin, GPIO.LOW)
    GPIO.cleanup()

Mocking Adafruit_DHT and RPi.GPIO for Windows
initialization...
saving membership...
done
program is ready for making decision based fuzzy logic
sensing...
Sensing: Temperature=25.0*C Humidity=80.0%
Normalization: Temperature=0.4 Humidity=0.8
fuzzy membership: Temperature={'cold': 0.9618841763091945, 'warm': 0.009414072602750017, 'hot': 1.3878009889412419e-09} Humidity={'low': 1.0, 'high': 6.78271149864835e-09}
5.539509341251452
turn on a machine
sensing...
Sensing: Temperature=25.0*C Humidity=80.0%
Normalization: Temperature=0.4 Humidity=0.8
fuzzy membership: Temperature={'cold': 0.9618841763091945, 'warm': 0.009414072602750017, 'hot': 1.3878009889412419e-09} Humidity={'low': 1.0, 'high': 6.78271149864835e-09}
5.539509341251452
a machine already turn on
sensing...
Sensing: Temperature=25.0*C Humidity=80.0%
Normalization: Temperature=0.4 Humidity=0.8
fuzzy membership: Temperature={'cold': 0.9618841763091945, 'warm': 0.009414072602750017, 'hot': 1.3878009889412419e-09} Humidity={'low': 